In [36]:
import pandas as pd

df = pd.read_excel("emotion_corpus.xls")

In [37]:
df = df[["감정_대분류","사람문장1"]]

In [38]:
df.dropna(inplace = True)

In [39]:
df

,감정_대분류,사람문장1
0,분노,일은 왜 해도 해도 끝이 없을까? 화가 난다.
1,분노,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.
2,분노,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...
3,분노,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...
4,분노,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.
...,...,...
51625,분노,나이가 먹고 이제 돈도 못 벌어 오니까 어떻게 살아가야 할지 막막해. 능력도 없고.
51626,불안,몸이 많이 약해졌나 봐. 이제 전과 같이 일하지 못할 것 같아 너무 짜증 나.
51627,상처,이제 어떻게 해야 할지 모르겠어. 남편도 그렇고 노후 준비도 안 되어서 미래가 걱정돼.
51628,불안,몇십 년을 함께 살았던 남편과 이혼했어. 그동안의 세월에 배신감을 느끼고 너무 화가 나.


In [40]:
# !pip install konlpy

from konlpy.tag import Okt
okt = Okt()
df['사람문장1'] = df['사람문장1'].map(lambda x: ' '.join(okt.morphs(x, stem = True)))

In [53]:
df.loc[(df["감정_대분류"] == "분노"),"감정_대분류"] = 0
df.loc[(df["감정_대분류"] == "기쁨"),"감정_대분류"] = 1
df.loc[(df["감정_대분류"] == "불안"),"감정_대분류"] = 2
df.loc[(df["감정_대분류"] == "당황"),"감정_대분류"] = 3
df.loc[(df["감정_대분류"] == "슬픔"),"감정_대분류"] = 4
df.loc[(df["감정_대분류"] == "상처"),"감정_대분류"] = 5
df.columns=["label","document"]
df
# df["감정_대분류"].unique()

,label,document
0,0,일 은 왜 해도 해도 끝 이 없다 ? 화가 나다 .
1,0,이번 달 에 또 급여 가 깎다 ! 물가 는 오르다 월급 만 자꾸 깎다 너무 화가 나 .
2,0,회사 에 신입 이 들어오다 말투 가 거슬리다 . 그렇다 애 를 매일 보다 하다 생각...
3,0,직장 에서 막내 라는 이유 로 나 에게만 온갖 심부름 을 시키다 . 일도 많다 데 ...
4,0,얼마 전 입사 한 신입사원 이 나르다 무시 하다 것 같다 너무 화가 나 .
...,...,...
51625,0,나이 가 먹다 이제 돈 도 못 벌다 오 니까 어떻다 살아가다 하다 막막하다 . 능력...
51626,2,몸 이 많이 약해지다 보다 . 이제 전과 같이 일 하다 못 하다 것 같다 너무 짜증...
51627,5,이제 어떻다 하다 하다 모르다 . 남편 도 그렇다 노후 준비 도 안 되어다 미래 가...
51628,2,몇십 년 을 함께 살다 남편 과 이혼 하다 . 그동안 의 세월 에 배신 감 을 느끼...


In [54]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_texts = train_df['document'].astype(str).tolist() # 문자열 데이터로 명시 후 리스트 화
train_labels = train_df['label'].tolist()
test_texts = test_df['document'].astype(str).tolist()
test_labels = test_df['label'].tolist()

In [55]:
print(len(train_df), len(test_df))

41304 10326


In [85]:
!pip install transformers

from transformers import BertTokenizer, BertForSequenceClassification

model_name = 'monologg/kobert'
tokenizer = BertTokenizer.from_pretrained(model_name)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [70]:
import torch
from torch.utils.data import DataLoader, Dataset

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings, test_labels)
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [71]:
model = BertForSequenceClassification.from_pretrained('monologg/kobert', num_labels=6)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
num_epochs = 5
learning_rate = 4e-5 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = torch.nn.CrossEntropyLoss()
model.to(device) 

for epoch in range(num_epochs):
    model.train() 
    total_loss = 0

    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Average Loss: {average_loss:.4f}")

  0%|          | 0/646 [00:00<?, ?it/s]

Epoch 1/5 - Average Loss: 0.7332


  0%|          | 0/646 [00:00<?, ?it/s]

Epoch 2/5 - Average Loss: 0.6632


  0%|          | 0/646 [00:00<?, ?it/s]

Epoch 3/5 - Average Loss: 0.5842


  0%|          | 0/646 [00:00<?, ?it/s]

Epoch 4/5 - Average Loss: 0.5171


  0%|          | 0/646 [00:00<?, ?it/s]

Epoch 5/5 - Average Loss: 0.4598


In [102]:
model.eval()
correct_predictions = 0
total_predictions = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        _, predicted_labels = torch.max(outputs.logits, dim=1)

        correct_predictions += torch.sum(predicted_labels == labels).item()
        total_predictions += labels.size(0)

accuracy = correct_predictions / total_predictions
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.3258


In [103]:
input_text = '진짜진짜 행복하다 너무 좋아'
input_encoding = tokenizer.encode_plus(
    input_text,
    truncation=True,
    padding=True,
    return_tensors='pt'
)

input_ids = input_encoding['input_ids'].to(device)
attention_mask = input_encoding['attention_mask'].to(device)

model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    _, predicted_labels = torch.max(outputs.logits, dim=1)
predicted_labels = predicted_labels.item()

print(predicted_labels)

4


"분노" = 0
"기쁨" = 1
"불안" = 2
"슬픔" = 4
"상처" = 5

In [100]:
from collections import Counter
Counter(df.iloc[:41304,0])

Counter({0: 7100, 1: 6126, 2: 7338, 3: 6669, 4: 7062, 5: 7009})